### Required libraries

In [105]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import csv
import pandas as pd
import sys
import numpy as np
import os.path as op
from os import mkdir, makedirs, getcwd
import scipy.stats as stats
import nipype.interfaces.fsl as fsl
from subprocess import call, Popen, check_output, CalledProcessError
import nibabel as nib
from shutil import copyfile, rmtree
import scipy.io as sio
from sklearn import cross_validation
from sklearn import linear_model
from numpy.polynomial.legendre import Legendre
import shlex
from scipy import signal
import operator
import gzip
from nilearn.signal import clean
from nilearn.image import smooth_img
from nilearn.input_data import NiftiMasker
import scipy.linalg as linalg
import string
import random
import xml.etree.cElementTree as ET
from time import localtime, strftime, sleep
from scipy.fftpack import fft, dct

### Utils

In [146]:
# regressors: to filter, no. time points x no. regressors
def filter_regressors(regressors, filtering, nTRs, TR):
    if len(filtering==0):
        print 'Error! Missing or wrong filtering flavor. Regressors were not filtered.'
    else:
        if filtering[0] == 'Butter':
            regressors = clean(regressors, detrend=False, standardize=False, 
                                  t_r=TR, high_pass=filtering[1], low_pass=filtering[2]).T
        elif filtering[0] == 'Gaussian':
            w = signal.gaussian(11,std=filtering[1])
            regressors = signal.lfilter(w,1,regressors, axis=0)  
    return regressors
    
def regress(niiImg, nTRs, TR, regressors, keepMean):
    X  = np.concatenate((np.ones([nTRs,1]), regressors), axis=1)
    N = niiImg.shape[0]
    for i in range(N):
        fit = np.linalg.lstsq(X, niiImg[i,:].T)[0]
        fittedvalues = np.dot(X, fit)
        resid = niiImg[i,:] - np.ravel(fittedvalues)
        if keepMean:
            niiImg[i,:] = X[:,0]*fit[0] + resid
        else:
            niiImg[i,:] = resid
    return niiImg     

def normalize(niiImg,flavor):
    if flavor == 'zscore':
        niiImg = stats.zscore(niiImg, axis=1, ddof=1)
        return niiImg
    elif flavor == 'pcSigCh':
        niiImg = 100 * (niiImg - np.mean(niiImg,axis=1)[:,np.newaxis]) / np.mean(niiImg,axis=1)[:,np.newaxis]
    else:
        print 'Warning! Wrong normalization flavor. Nothing was done'
    return niiImg    

def legendre_poly(order, nTRs):
    # ** a) create polynomial regressor **
    x = np.arange(nTRs)
    x = x - x.max()/2
    num_pol = range(order+1)
    y = np.ones((len(num_pol),len(x)))   
    coeff = np.eye(order+1)
    # Print out text file for each polynomial to be used as a regressor
    for i in num_pol:
        myleg = Legendre(coeff[i])
        y[i,:] = myleg(x) 
        if i>0:
            y[i,:] = y[i,:] - np.mean(y[i,:])
            y[i,:] = y[i,:]/np.max(y[i,:])
        np.savetxt(op.join(buildpath(subject,fmriRun),
                           'poly_detrend_legendre' + str(i) + '.txt'), y[i,:] ,fmt='%.4f')
    return y

def load_img(fmriFile, maskAll):
    if isCifti:
        toUnzip = fmriFile.replace('_Atlas.dtseries.nii','.nii.gz')
        cmd = 'wb_command -cifti-convert -to-text {} {}'.format(fmriFile,op.join(buildpath(subject,fmriRun),'.tsv'))
        call(cmd,shell=True)
    else:
        toUnzip = fmriFile

    with open(toUnzip, 'rb') as fFile:
        decompressedFile = gzip.GzipFile(fileobj=fFile)
        outFilePath = op.join(buildpath(subject, fmriRun), fmriRun+'.nii')
        with open(outFilePath, 'wb') as outfile:
            outfile.write(decompressedFile.read())

    volFile = outFilePath

    img = nib.load(volFile)
    
    myoffset = img.header.sizeof_hdr + 4 + img.header.get_data_offset()
    data = np.memmap(volFile, dtype=img.header.get_data_dtype(), mode='c', order='F',
                     offset=myoffset,shape=img.header.get_data_shape())

    nRows, nCols, nSlices, nTRs = img.header.get_data_shape()
    TR = img.header.structarr['pixdim'][4]
    niiImg = data.reshape([nRows*nCols*nSlices, nTRs], order='F')
    niiImg = niiImg[maskAll,:]
    return niiImg, nRows, nCols, nSlices, nTRs, img.affine, TR

def plot_hist(score,title,xlabel):
    h,b = np.histogram(score, bins='auto')
    plt.hist(score,bins=b)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Frequency')
    return h

def makeTissueMasks(subject,fmriRun,overwrite):
    fmriFile = op.join(buildpath(subject,fmriRun), fmriRun+suffix+'.nii.gz')
    WMmaskFileout = op.join(buildpath(subject,fmriRun), 'WMmask.nii')
    CSFmaskFileout = op.join(buildpath(subject,fmriRun), 'CSFmask.nii')
    GMmaskFileout = op.join(buildpath(subject,fmriRun), 'GMmask.nii')
    
    if not op.isfile(GMmaskFileout) or overwrite:
        # load ribbon.nii.gz and wmparc.nii.gz
        ribbonFilein = op.join(DATADIR, subject, 'MNINonLinear','ribbon.nii.gz')
        wmparcFilein = op.join(DATADIR, subject, 'MNINonLinear', 'wmparc.nii.gz')
        # make sure it is resampled to the same space as the functional run
        ribbonFileout = op.join(buildpath(subject,fmriRun), 'ribbon.nii.gz')
        wmparcFileout = op.join(buildpath(subject,fmriRun), 'wmparc.nii.gz')
        # make identity matrix to feed to flirt for resampling
        with open('eye.mat','w') as fid:
            fid.write('1 0 0 0\n0 1 0 0\n0 0 1 0\n0 0 0 1')
        
        flirt_ribbon = fsl.FLIRT(in_file=ribbonFilein, out_file=ribbonFileout,\
            reference=fmriFile, apply_xfm=True,\
            in_matrix_file='eye.mat', interp='nearestneighbour')
        flirt_ribbon.run()

        flirt_wmparc = fsl.FLIRT(in_file=wmparcFilein, out_file=wmparcFileout,\
            reference=fmriFile, apply_xfm=True,\
            in_matrix_file='eye.mat', interp='nearestneighbour')
        flirt_wmparc.run()
        
        # load nii (ribbon & wmparc)
        ribbon = nib.load(ribbonFileout).get_data()
        wmparc = nib.load(wmparcFileout).get_data()
        
        # white & CSF matter mask
        # indices are from FreeSurferColorLUT.txt
        
        # Left-Cerebral-White-Matter, Right-Cerebral-White-Matter
        ribbonWMstructures = [2, 41]
        # Left-Cerebral-Cortex, Right-Cerebral-Cortex
        ribbonGMstrucures = [3, 42]
        # Cerebellar-White-Matter-Left, Brain-Stem, Cerebellar-White-Matter-Right
        wmparcWMstructures = [7, 16, 46]
        # Left-Cerebellar-Cortex, Right-Cerebellar-Cortex, Thalamus-Left, Caudate-Left
        # Putamen-Left, Pallidum-Left, Hippocampus-Left, Amygdala-Left, Accumbens-Left 
        # Diencephalon-Ventral-Left, Thalamus-Right, Caudate-Right, Putamen-Right
        # Pallidum-Right, Hippocampus-Right, Amygdala-Right, Accumbens-Right
        # Diencephalon-Ventral-Right
        wmparcGMstructures = [8, 47, 10, 11, 12, 13, 17, 18, 26, 28, 49, 50, 51, 52, 53, 54, 58, 60]
        # Fornix, CC-Posterior, CC-Mid-Posterior, CC-Central, CC-Mid-Anterior, CC-Anterior
        wmparcCCstructures = [250, 251, 252, 253, 254, 255]
        # Left-Lateral-Ventricle, Left-Inf-Lat-Vent, 3rd-Ventricle, 4th-Ventricle, CSF
        # Left-Choroid-Plexus, Right-Lateral-Ventricle, Right-Inf-Lat-Vent, Right-Choroid-Plexus
        wmparcCSFstructures = [4, 5, 14, 15, 24, 31, 43, 44, 63]
        
        # make masks
        WMmask = np.double(np.logical_and(np.logical_and(np.logical_or(np.logical_or(np.in1d(ribbon, ribbonWMstructures),
                                                                              np.in1d(wmparc, wmparcWMstructures)),
                                                                np.in1d(wmparc, wmparcCCstructures)),
                                                  np.logical_not(np.in1d(wmparc, wmparcCSFstructures))),
                                   np.logical_not(np.in1d(wmparc, wmparcGMstructures))))
        CSFmask = np.double(np.in1d(wmparc, wmparcCSFstructures))
        GMmask = np.double(np.logical_or(np.in1d(ribbon,ribbonGMstrucures),np.in1d(wmparc,wmparcGMstructures)))
        
        # write masks
        ref = nib.load(wmparcFileout)
        WMmask = np.reshape(WMmask,ref.shape)
        img = nib.Nifti1Image(WMmask, ref.affine)
        nib.save(img, WMmaskFileout)
        
        CSFmask = np.reshape(CSFmask,ref.shape)
        img = nib.Nifti1Image(CSFmask, ref.affine)
        nib.save(img, CSFmaskFileout)
        
        GMmask = np.reshape(GMmask,ref.shape)
        img = nib.Nifti1Image(GMmask, ref.affine)
        nib.save(img, GMmaskFileout)
        
        # delete temporary files
        cmd = 'rm eye.mat ribbon_flirt.mat wmparc_flirt.mat'
        call(cmd,shell=True)
        
        
    tmpnii = nib.load(WMmaskFileout)
    myoffset = tmpnii.header.sizeof_hdr + 4 + tmpnii.header.get_data_offset()
    data = np.memmap(WMmaskFileout, dtype=tmpnii.header.get_data_dtype(), mode='r', order='F',
                     offset=myoffset,shape=tmpnii.header.get_data_shape())
    nRows, nCols, nSlices = tmpnii.header.get_data_shape()
    maskWM = np.reshape(data > 0,nRows*nCols*nSlices, order='F')
    del data
    tmpnii = nib.load(CSFmaskFileout)
    myoffset = tmpnii.header.sizeof_hdr + 4 + tmpnii.header.get_data_offset()
    data = np.memmap(CSFmaskFileout, dtype=tmpnii.header.get_data_dtype(), mode='r', order='F', 
                     offset=myoffset,shape=tmpnii.header.get_data_shape())
    maskCSF = np.reshape(data > 0,nRows*nCols*nSlices, order='F')
    del data
    tmpnii = nib.load(GMmaskFileout)
    myoffset = tmpnii.header.sizeof_hdr + 4 + tmpnii.header.get_data_offset()
    data = np.memmap(GMmaskFileout, dtype=tmpnii.header.get_data_dtype(), mode='r', order='F', 
                     offset=myoffset,shape=tmpnii.header.get_data_shape())
    maskGM = np.reshape(data > 0,nRows*nCols*nSlices, order='F')
    del data
    maskAll = np.logical_or(np.logical_or(maskWM, maskCSF), maskGM)
    maskWM_ = maskWM[maskAll]
    maskCSF_ = maskCSF[maskAll]
    maskGM_ = maskGM[maskAll]
    
    return maskAll, maskWM_, maskCSF_, maskGM_


def extract_noise_components(niiImg, num_components=5, extra_regressors=None):
    """Largely based on https://github.com/nipy/nipype/blob/master/examples/
    rsfmri_vol_surface_preprocessing_nipy.py#L261
    Derive components most reflective of physiological noise according to
    aCompCor method (Behzadi 2007)
    Parameters
    ----------
    niiImg: raw data
    num_components: number of components to use for noise decomposition
    extra_regressors: additional regressors to add
    Returns
    -------
    components: n_time_points x regressors
    """
    niiImgWMCSF = niiImg[np.logical_or(maskWM_,maskCSF_),:] 
    
    niiImgWMCSF[np.isnan(np.sum(niiImgWMCSF, axis=1)), :] = 0
    # remove mean and normalize by variance
    # voxel_timecourses.shape == [nvoxels, time]
    X = niiImgWMCSF.T
    stdX = np.std(X, axis=0)
    stdX[stdX == 0] = 1.
    stdX[np.isnan(stdX)] = 1.
    stdX[np.isinf(stdX)] = 1.
    X = (X - np.mean(X, axis=0)) / stdX
    u, _, _ = linalg.svd(X, full_matrices=False)
    components = u[:, :num_components]
    
    if extra_regressors:
        components = np.hstack((components, regressors))

    return components

def conf2XML(inFile, dataDir, operations, startTime, endTime, fname):
    doc = ET.Element("pipeline")
    
    nodeInput = ET.SubElement(doc, "input")
    nodeInFile = ET.SubElement(nodeInput, "inFile")
    nodeInFile.text = inFile
    nodeDataDir = ET.SubElement(nodeInput, "dataDir")
    nodeDataDir.text = dataDir
    
    nodeDate = ET.SubElement(doc, "date")
    nodeDay = ET.SubElement(nodeDate, "day")
    day = strftime("%Y-%m-%d", localtime())
    nodeDay.text = day
    stime = strftime("%H:%M:%S", startTime)
    etime = strftime("%H:%M:%S", endTime)
    nodeStart = ET.SubElement(nodeDate, "timeStart")
    nodeStart.text = stime
    nodeEnd = ET.SubElement(nodeDate, "timeEnd")
    nodeEnd.text = etime
    
    nodeSteps = ET.SubElement(doc, "steps")
    for op in operations:
        if op[1] == 0: continue
        nodeOp = ET.SubElement(nodeSteps, "operation", name=op[0])
        nodeOrder = ET.SubElement(nodeOp, "order")
        nodeOrder.text = str(op[1])
        nodeFlavor = ET.SubElement(nodeOp, "flavor")
        nodeFlavor.text = str(op[2])
    tree = ET.ElementTree(doc)
    tree.write(fname)
    
def fnSubmitToCluster(strScript, strJobFolder, strJobUID, resources):
    specifyqueue = ''
    # clean up .o and .e
    tmpfname = op.join(strJobFolder,strJobUID)
    if op.isfile(tmpfname+'.e'): remove(tmpfname+'.e')       
    if op.isfile(tmpfname+'.o'): remove(tmpfname+'.o')       
   
    strCommand = 'qsub {} -cwd -V {} -N {} -e "{}" -o "{}" "{}"'.format(specifyqueue,resources,strJobUID,
                      op.join(strJobFolder,strJobUID+'.e'), op.join(strJobFolder,strJobUID+'.o'), strScript)
    # write down the command to a file in the job folder
    with open(op.join(strJobFolder,strJobUID+'.cmd'),'w+') as hFileID:
        hFileID.write(strCommand+'\n')
    # execute the command
    cmdOut = check_output(strCommand, shell=True)
    return cmdOut.split()[2]    

def _interpolate(a, b, fraction):
    """Returns the point at the given fraction between a and b, where
    'fraction' must be between 0 and 1.
    """
    return a + (b - a)*fraction;

def scoreatpercentile(a, per, limit=(), interpolation_method='fraction'):
    """
    This function is grabbed from scipy

    """
    values = np.sort(a, axis=0)
    if limit:
        values = values[(limit[0] <= values) & (values <= limit[1])]

    idx = per /100. * (values.shape[0] - 1)
    if (idx % 1 == 0):
        score = values[idx]
    else:
        if interpolation_method == 'fraction':
            score = _interpolate(values[int(idx)], values[int(idx) + 1],
                                 idx % 1)
        elif interpolation_method == 'lower':
            score = values[np.floor(idx)]
        elif interpolation_method == 'higher':
            score = values[np.ceil(idx)]
        else:
            raise ValueError("interpolation_method can only be 'fraction', " \
                             "'lower' or 'higher'")

    return score

def dctmtx(N):
    K=N
    n = range(N)
    C = np.zeros((len(n), K))
    C[:,0] = np.ones((len(n)))/np.sqrt(N)
    doublen = [2*x+1 for x in n]
    for k in range(1,K):
        C[:,k] = np.sqrt(2/N)*np.cos([np.pi*x*(k-1)/(2*N) for x in doublen])        
    return C 

### Parameters

In [147]:
behavFile = 'unrestricted_luckydjuju_11_17_2015_0_47_11.csv'
release = 'Q2'
outScore = 'PMAT24_A_CR'
pipelineName = 'Finn'
parcellation = 'shenetal_neuroimage2013'
overwrite = False
thisRun = 'rfMRI_REST1'
isDataClean = True
doPlot = True
queue = False
normalize = 'zscore'
isCifti = False
keepMean = False
class config(object):
    filtering = []
    doScrubbing = False

# customize path to get access to single runs
def buildpath(subject,fmriRun):
    return op.join(DATADIR, subject,'MNINonLinear','Results',fmriRun)

In [84]:
# these functions allow Paola & Julien to run code locally with their own path definitions
def getDataDir(x):
    return {
        'esplmatlabw02.csmc.edu': '/home/duboisjx/vault/data/HCP/MRI',
        'sculpin.caltech.edu': '/data/jdubois/data/HCP/MRI',
    }.get(x, '/media/paola/HCP/')    # /media/paola/HCP is default if x not found
def getParcelDir(x):
    return {
        'esplmatlabw02.csmc.edu': '/home/duboisjx/vault/data/parcellations/',
        'sculpin.caltech.edu': '/data/jdubois/data/parcellations/',
    }.get(x, '/data/pgaldi/parcellations/')    # /home/paola/parcellations/ is default if x not found
import socket
HOST=socket.gethostname()
DATADIR=getDataDir(HOST)
PARCELDIR=getParcelDir(HOST)

#DATADIR = '/media/paola/HCP/'
#PARCELDIR = '/home/paola/parcellations'

if queue: priority=-100

if thisRun == 'rfMRI_REST1':
    outMat = 'rest_1_mat'
elif thisRun == 'rfMRI_REST2':
    outMat = 'rest_1_mat'
else:
    sys.exit("Invalid run code")  
    
suffix = '_hp2000_clean' if isDataClean else ''   

In [101]:
subject = '734045'
fmriRun = 'rfMRI_REST1_LR'
fmriFile = op.join(buildpath(subject,fmriRun),'rfMRI_REST1_LR.nii.gz')

### Pipeline definition

The pipeline workflow is defined by two dictionaries. 

The struct <b>Operations</b> encodes the order of generic pipeline steps, with 0 for skipping an operation, and otherwise a number indicating when the operation should be performed. Note that several operations may have the same position (e.g., motion regression and tissue regression may both have order = 3, which means they should be performed in the same regression). For each operation an array encodes the flavor of each step and parameters when needed.

#### Finn's pipeline

In [132]:
Operations= [
    ['VoxelNormalization',      1, ['zscore']],
    ['Detrending',              2, ['legendre', 3, 'WMCSF']],
    ['TissueRegression',        3, ['WMCSF']],
    ['MotionRegression',        4, ['[R dR]']],
    ['TemporalFiltering',       5, ['Gaussian', 1]],
    ['Detrending',              6, ['legendre', 3,'GM']],
    ['GlobalSignalRegression',  7, []],
    ['Scrubbing',               0, ['fd', 0.2]],
    ['SpatialSmoothing',        0, ['Gaussian', 6]],
    ['ICAdenoising',            0, ['ICAFIX']],
]

### Pipeline setup

Every step is associated with a function.

In [149]:
def MotionRegression(niiImg, flavor, masks, imgInfo):
    # assumes that data is organized as in the HCP
    motionFile = op.join(buildpath(subject,fmriRun), 'Movement_Regressors_dt.txt')
    data = np.genfromtxt(motionFile)
    if flavor[0] == 'R':
        X = data[:,:6]
    elif flavor[0] == 'R dR':
        X = data
    elif flavor[0] == 'R R^2':
        data_squared = data ** 2
        X = np.concatenate((data, data_squared), axis=1)
    elif flavor[0] == 'R R^2 R-1 R-1^2':
        data_roll = np.roll(data, 1, axis=0)
        data_squared = data ** 2
        data_roll[0] = 0
        data_roll_squared = data_roll ** 2
        X = np.concatenate((data, data_squared, data_roll, data_roll_squared), axis=1)
    elif flavor[0] == 'R R^2 R-1 R-1^2 R-2 R-2^2':
        data_roll = np.roll(data, 1, axis=0)
        data_squared = data ** 2
        data_roll[0] = 0
        data_roll_squared = data_roll ** 2
        data_roll2 = np.roll(data_roll, 1, axis=0)
        data_roll2[0] = 0
        data_roll2_squared = data_roll2 ** 2
        X = np.concatenate((data, data_squared, data_roll, data_roll_squared, data_roll2, data_roll2_squared), axis=1)
    else:
        'Wrong flavor, using default regressors: R dR'
        X = data
        
    # if filtering has already been performed, regressors need to be filtered too
    if len(config.filtering)>0:
        nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
        X = filter_regressors(X, config.filtering, nTRs, TR)  
        
    if config.doScrubbing:
        toCensor = np.loadtxt(op.join(buildpath(subject,fmriRun), 'Censored_TimePoints.txt'), dtype=np.dtype(np.int32))
        toReg = np.zeros((nTRs, 1))
        toReg[toCensor] = 1
        X = np.concatenate((X, toReg), axis=1)
        
    return X

def Scrubbing(niiImg, flavor, masks, imgInfo):
    thr = flavor[1]
    if flavor[0] == 'DVARS':
        # pcSigCh
        niiImg = 100 * niiImg / np.mean(niiImg,axis=1)[:,np.newaxis] -100
        dt = np.diff(niiImg, n=1, axis=1)
        dt = np.concatenate((np.zeros((dt.shape[0],1)), dt), axis=1)
        score = np.sqrt(np.mean(dt**2,0))        
    elif flavor[0] == 'FD':
        motionFile = op.join(buildpath(subject,fmriRun), 'Movement_Regressors_dt.txt')
        dmotpars = np.abs(np.genfromtxt(motionFile)[:,6:]) #derivatives
        headradius=50 #50mm as in Powers et al.
        disp=dmotpars.copy()
        disp[:,3:]=np.pi*headradius*2*(disp[:,3:]/360)
        score=np.sum(disp,1)
    else:
        print 'Wrong scrubbing flavor. Nothing was done'
        return niiImg
    censored = np.where(score>thr)
    np.savetxt(op.join(buildpath(subject,fmriRun), 'Censored_TimePoints.txt'), delimiter='\n', fmt='%d')
    config.doScrubbing = True
    return niiImg

def TissueRegression(niiImg, flavor, masks, imgInfo):
    maskAll, maskWM_, maskCSF_, maskGM_ = masks
    nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    
    if isCifti:
        niiImgGM = niiImg
    else:
        niiImgGM = niiImg[maskGM_,:]
        
    if flavor[0] == 'CompCor':
        X = extract_noise_components(niiImg, num_components=flavor[1])
        niiImgGM = regress(niiImgGM, nTRs, TR, X, keepMean)
    elif flavor[0] == 'WMCSF':
        meanWM = np.mean(np.float64(niiImg[maskWM_,:]),axis=0)
        meanWM = meanWM - np.mean(meanWM)
        meanWM = meanWM/max(meanWM)
        meanCSF = np.mean(np.float64(niiImg[maskCSF_,:]),axis=0)
        meanCSF = meanCSF - np.mean(meanCSF)
        meanCSF = meanCSF/max(meanCSF)
        X  = np.concatenate((meanWM[:,np.newaxis], meanCSF[:,np.newaxis]), axis=1)
        niiImgGM = regress(niiImgGM, nTRs, TR, X, keepMean)
    elif flavor[0] == 'WMCSF+dt':
        meanWM = np.mean(np.float64(niiImg[maskWM_,:]),axis=0)
        meanWM = meanWM - np.mean(meanWM)
        meanWM = meanWM/max(meanWM)
        meanCSF = np.mean(np.float64(niiImg[maskCSF_,:]),axis=0)
        meanCSF = meanCSF - np.mean(meanCSF)
        meanCSF = meanCSF/max(meanCSF)
        dtWM=np.zeros(meanWM.shape)
        dtWM[1:] = np.diff(meanWM, n=1)
        dtCSF=np.zeros(meanCSF.shape)
        dtCSF[1:] = np.diff(meanCSF, n=1)
        X  = np.concatenate((meanWM[:,np.newaxis], meanCSF[:,np.newaxis], 
                             dtWM[:,np.newaxis], dtCSF[:,np.newaxis]), axis=1)
        niiImgGM = regress(niiImgGM, nTRs, TR, X, keepMean)
    else:
        print 'Warning! Wrong tissue regression flavor. Nothing was done'
    if not isCifti:
        niiImg[maskGM_,:] = niiImgGM
    else:
        niiImg = niiImgGM
    return niiImg

def Detrending(niiImg, flavor, masks, imgInfo):
    maskAll, maskWM_, maskCSF_, maskGM_ = masks
    nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    
    if flavor[2] == 'WMCSF':
        niiImgWMCSF = niiImg[np.logical_or(maskWM_,maskCSF_),:]
        if flavor[0] == 'legendre':
            y = legendre_poly(flavor[1],nTRs)                
            niiImgWMCSF = regress(niiImgWMCSF, nTRs, TR, y[1:4,:].T, keepMean)
        elif flavor[0] == 'poly':       
            x = np.arange(nTRs)
            nPoly = flavor[0] + 1
            y = np.ones((nPoly,len(x)))
            for i in range(nPoly):
                y[i,:] = (x - (np.max(x)/2)) **(i+1)
                y[i,:] = y[i,:] - np.mean(y[i,:])
                y[i,:] = y[i,:]/np.max(y[i,:]) 
            niiImgWMCSF = regress(niiImgWMCSF, nTRs, TR, y[1:4,:].T, keepMean)
        else:
            print 'Warning! Wrong detrend flavor. Nothing was done'
        niiImg[np.logical_or(maskWM_,maskCSF_),:] = niiImgWMCSF    
    elif flavor[2] == 'GM':
        if isCifti:
            niiImgGM = niiImg
        else:
            niiImgGM = niiImg[maskGM_,:]

        if flavor[0] == 'legendre':
            y = legendre_poly(flavor[1], nTRs)
            niiImgGM = regress(niiImgGM, nTRs, TR, y[1:4,:].T, keepMean)
        elif flavor[0] == 'poly':       
            x = np.arange(nTRs)
            nPoly = flavor[0] + 1
            y = np.ones((nPoly,len(x)))
            for i in range(nPoly):
                y[i,:] = (x - (np.max(x)/2)) **(i+1)
                y[i,:] = y[i,:] - np.mean(y[i,:])
                y[i,:] = y[i,:]/np.max(y[i,:])
            niiImgGM = regress(niiImgGM, nTRs, TR, y[1:4,:].T, keepMean)
        else:
            print 'Warning! Wrong detrend flavor. Nothing was done'

        if not isCifti:
            niiImg[maskGM_,:] = niiImgGM
        else:
            niiImg = niiImgGM
    else:
        print 'Warning! Wrong detrend mask. Nothing was done' 
    return niiImg     

   

def SpatialSmoothing(niiImg, flavor, masks, imgInfo):
    maskAll, maskWM_, maskCSF_, maskGM_ = masks
    nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    
    niiimg = np.zeros((nRows*nCols*nSlices, nTRs))
    niiimg[maskAll,:] = niiImg
    niiimg = np.reshape(niiimg, (nRows, nCols, nSlices, nTRs), order='F')
    newimg = nib.Nifti1Image(niiimg, affine)
    if flavor[0] == 'Gaussian':
        newimg = smooth_img(newimg, flavor[1])
        niiimg = np.reshape(newimg.get_data(), (nRows*nCols*nSlices, nTRs), order='F')
        niiImg = niiimg[maskAll,:]
    elif flavor[0] == 'GaussianGM':
        GMmaskFile = op.join(buildpath(subject,fmriRun),'GMmask.nii')
        NiftiMasker(mask_img=GMmaskFile, sessions=None, smoothing_fwhm=flavor[1])
        niiImg[maskGM_,:] = masker.fit_transform(newimg).T
    else:
        print 'Warning! Wrong smoothing flavor. Nothing was done'
    return niiImg  

def TemporalFiltering(niiImg, flavor, masks, imgInfo):
    maskAll, maskWM_, maskCSF_, maskGM_ = masks
    nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    
    if flavor[0] == 'Butter':
        niiImg = clean(niiImg.T, detrend=False, standardize=False, 
                              t_r=TR, high_pass=flavor[1], low_pass=flavor[2]).T
    elif flavor[0] == 'Gaussian':
        w = signal.gaussian(11,std=flavor[1])
        niiImg = signal.lfilter(w,1,niiImg)
    elif flavor[0] == 'DCT':
        K = dctmtx(nTRs)
        HPC = 1/flavor[1]
        LPC = 1/flavor[2]
        nHP = np.fix(2*(nTRs*TR)/HPC + 1)
        nLP = np.fix(2*(nTRs*TR)/LPC + 1)
        K = K[:,np.concatenate((range(1,nHP),range(int(nLP)-1,nTRs)))]
        return K
    else:
        print 'Warning! Wrong temporal filtering flavor. Nothing was done'    
        return niiImg
    config.filtering = flavor
    return niiImg    
    
def ICAdenoising(niiImg, flavor, masks, imgInfo):
    maskAll, maskWM_, maskCSF_, maskGM_ = masks
    nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    print 'ICAdenoising : '+flavor
    print 'Warning! Method not implemented yet, nothing was done'
    return niiImg

def GlobalSignalRegression(niiImg, flavor, masks, imgInfo):
    meanAll = np.mean(niiImg,axis=0)
    meanAll = meanAll - np.mean(meanAll)
    meanAll = meanAll/max(meanAll)
    return meanAll[:,np.newaxis]

def VoxelNormalization(niiImg, flavor, masks, imgInfo):
    if flavor[0] == 'zscore':
        niiImg = stats.zscore(niiImg, axis=1, ddof=1)
        return niiImg
    elif flavor[0] == 'pcSigCh':
        niiImg = 100 * (niiImg - np.mean(niiImg,axis=1)[:,np.newaxis]) / np.mean(niiImg,axis=1)[:,np.newaxis]
    else:
        print 'Warning! Wrong normalization flavor. Nothing was done'
    return niiImg  


Hooks={
    'MotionRegression'       : MotionRegression,
    'Scrubbing'              : Scrubbing,
    'TissueRegression'       : TissueRegression,
    'Detrending'             : Detrending,
    'SpatialSmoothing'       : SpatialSmoothing,
    'TemporalFiltering'      : TemporalFiltering,  
    'ICAdenoising'           : ICAdenoising,
    'GlobalSignalRegression' : GlobalSignalRegression,  
    'VoxelNormalization'     : VoxelNormalization,
}

sortedOperations = sorted(Operations, key=operator.itemgetter(1))
steps = {}
Flavors = {}
cstep = 0
for opr in sortedOperations:
    if opr[1]==0:
        continue
    else:
        if opr[1]!=cstep:
            cstep=cstep+1
            steps[cstep] = [opr[0]]
            Flavors[cstep] = [opr[2]]
        else:
            steps[cstep].append(opr[0])
            Flavors[cstep].append(opr[2])

Steps are executed sequentially. 

In case of regression, when multiple regression steps have the same order, all the regressors are combined and a single regression is executed (other operations are executed in order).

Tissue regression has a custom handling, since regression is constrained to GM voxels.

If filtering based on Discrete Cosine Transform (DCT) is involved, regression is used for band pass filtering.

In [143]:
nsteps = len(steps)
for i in range(1,nsteps+1):
    step = steps[i]
    print 'Step '+str(i)+' '+str(step[0])+' '+ str(Flavors[i][0])
    
    print str('Regression' in step[0] or ('TemporalFiltering' in step[0] and 'Gaussian' in Flavors[i][0]))

Step 1 VoxelNormalization ['zscore']
False
Step 2 Detrending ['legendre', 3, 'WMCSF']
False
Step 3 TissueRegression ['WMCSF']
True
Step 4 MotionRegression ['[R dR]']
True
Step 5 TemporalFiltering ['Gaussian', 1]
True
Step 6 Detrending ['legendre', 3, 'GM']
False
Step 7 GlobalSignalRegression []
True


In [ ]:
def runPipeline(subject, fmriRun, fmriFile):
    
    timeStart = localtime()
    
    print 'Step 0'
    print 'Building WM, CSF and GM masks...'
    masks = makeTissueMasks(subject,fmriRun,False)
    maskAll, maskWM_, maskCSF_, maskGM_ = masks

    print 'Loading data in memory...'
    imgInfo = load_img(fmriFile, maskAll)
    niiImg, nRows, nCols, nSlices, nTRs, affine, TR = imgInfo
    nsteps = len(steps)
    for i in range(1,nsteps+1):
        step = steps[i]
        print 'Step '+str(i)+' '+str(step[0])
        
        if len(step) == 1:
            # Atomic operations
            if 'Regression' in step[0] or ('TemporalFiltering' in step[0] and 'DCT' in Flavors[i][0]):
                if step[0]=='TissueRegression': #regression constrained to GM
                    niiImg = Hooks[step[0]](niiImg, Flavors[i][0], masks, imgInfo[1:])
                else:
                    r0 = Hooks[step[0]](niiImg, Flavors[i][0], masks, imgInfo[1:])
                    niiImg = regress(niiImg, nTRs, TR, r0, keepMean)
            else:
                niiImg = Hooks[step[0]](niiImg, Flavors[i][0], masks, imgInfo[1:])
        else:
            # When multiple regression steps have the same order, all the regressors are combined
            # and a single regression is performed (other operations are executed in order)
            r = np.empty((nTRs, 0))
            for j in range(len(step)):
                opr = step[j]
                if 'Regression' in opr or ('TemporalFiltering' in opr and 'DCT' in Flavors[i][j]):
                    if opr=='TissueRegression': #regression constrained to GM
                        niiImg = Hooks[opr](niiImg, Flavors[i][j], masks, imgInfo[1:])
                    else:    
                        r0 = Hooks[opr](niiImg, Flavors[i][j], masks, imgInfo[1:])
                        r = np.append(r, r0, axis=1)
                else:
                    niiImg = Hooks[opr](niiImg, Flavors[i][j], masks, imgInfo[1:])
            if r.shape[1] > 0:
                niiImg = regress(niiImg, nTRs, TR, r, keepMean)    
        niiImg[np.isnan(niiImg)] = 0

    print 'Done! Copy the resulting file...'
    rstring = ''.join(random.SystemRandom().choice(string.ascii_lowercase +string.ascii_uppercase + string.digits) for _ in range(8))
    outFile = fmriRun+'_'+rstring
    if isCifti:
        # write to text file
        np.savetxt(op.join(buildpath(subject,fmriRun),outfile+'.tsv'),niiImg, delimiter='\t', fmt='%.6f')
        # need to convert back to cifti
        cmd = 'wb_command -cifti-convert -from-text {} {} {}'.format(op.join(buildpath(subject,fmriRun),'.tsv'),
                                                                     fmriFile,
                                                                     op.join(buildpath(subject,fmriRun),outFile+'.dtseries.nii'))
        call(cmd,shell=True)
        # delete temporary files
        cmd = 'rm -r {}/*.tsv'.format(buildpath(subject,fmriRun))
        call(cmd,shell=True)
        del niiImg
    else:
        niiimg = np.zeros((nRows*nCols*nSlices, nTRs))
        niiimg[maskAll,:] = niiImg
        del niiImg
        niiimg = np.reshape(niiimg, (nRows, nCols, nSlices, nTRs), order='F')
        newimg = nib.Nifti1Image(niiimg, affine)
        nib.save(newimg,op.join(buildpath(subject,fmriRun),outFile+'.nii.gz'))
        del niiimg 

    timeEnd = localtime()  
    
    outXML = rstring+'.xml'
    conf2XML(fmriFile, DATADIR, sortedOperations, timeStart, timeEnd, op.join(buildpath(subject,fmriRun),outXML))
    
    print 'Preprocessing complete. Starting FC computation...'
    # After preprocessing, functional connectivity is computed
    ResultsDir = op.join(DATADIR,'Results'v
    if not op.isdir(ResultsDir): mkdir(ResultsDir)
    ResultsDir = op.join(ResultsDir,pipelineName)
    if not op.isdir(ResultsDir): mkdir(ResultsDir)
    ResultsDir = op.join(ResultsDir,parcellation)
    if not op.isdir(ResultsDir): mkdir(ResultsDir)
    if parcellation=='shenetal_neuroimage2013':
        uniqueParcels = range(268)
        isCifti = 0
        parcelVolume = 'fconn_atlas_150_2mm.nii'
    elif parcellation=='Glasser_Aseg_Suit':
        isCifti = 1
        parcelVolume = 'Parcels.dlabel.nii'
        uniqueParcels = range(405)
    else:
        print "Invalid parcellation code"
        return
    
    for iParcel in uniqueParcels:
        if not isCifti:
            parcelMaskFile = op.join(PARCELDIR,parcellation,'parcel{:03d}.nii.gz'.format(iParcel+1))
            if not op.isfile(parcelMaskFile):
                print 'Making a binary volume mask for each parcel'
                mymaths1 = fsl.maths.MathsCommand(in_file=op.join(PARCELDIR, parcellation,'fconn_atlas_150_2mm.nii'),\
                    out_file=parcelMaskFile, args='-thr {:.1f} -uthr {:.1f}'.format(iParcel+1-0.1, iParcel+1+0.1)) 
                mymaths1.run()
    if not op.isfile(fmriFile):
        print fmriFile, 'does not exist'
        return
    
    tsDir = op.join(buildpath(subject,fmriRun),parcellation)
    if not op.isdir(tsDir): mkdir(tsDir)
    alltsFile = op.join(ResultsDir,subject+'_'+fmriRun+'.txt')
    
    if not (op.isfile(alltsFile)) or overwrite:            
        # calculate signal in each of the nodes by averaging across all voxels/grayordinates in node
        print 'Extracting mean data from',str(len(uniqueParcels)),'parcels for ',outFile
       
        for iParcel in uniqueParcels:
            tsFile = op.join(tsDir,'parcel{:03d}.txt'.format(iParcel+1))
            if not op.isfile(tsFile):
                if not isCifti:
                    parcelMaskFile = op.join(PARCELDIR,parcellation,'parcel{:03d}.nii.gz'.format(iParcel+1))
                    
                    # simply average the voxels within the mask
                    meants1 = fsl.ImageMeants(in_file=op.join(buildpath(subject,fmriRun),outFile+'.nii.gz'), out_file=tsFile, mask=parcelMaskFile)
                    meants1.run()
                else:
                    # extract data in the parcel
                    parcelMaskFile = op.join(PARCELDIR,parcellation,'parcel{:03d}.dscalar.nii'.format(iParcel+1))
                    cmd = 'wb_command -cifti-label-to-roi {} {} -key {}'.format(
                        op.joinpath(PARCELDIR,parcellation,parcelVolume), parcelMaskFile,iParcel+1)
                    call(cmd,shell=True)
                    cmd = 'wb_command -cifti-roi-average {} {} -cifti-roi {}'.format(
                        op.join(buildpath(subject,fmriRun),outFile+'.nii.gz'),tsFile, parcelMaskFile)
                
        # concatenate all ts
        print 'Concatenating data'
        cmd = 'paste '+op.join(tsDir,'parcel*.txt')+' > '+alltsFile
        call(cmd, shell=True)

    
    

In [ ]:
runPipeline(subject, fmriRun, fmriFile)

### Get subjects

In [ ]:
df = pd.read_csv(behavFile)

# select subjects according to release
if release == 'Q2':
    ind = (df['Release'] == 'Q2') | (df['Release'] == 'Q1')
elif release == 'S500':
    ind = (df['Release'] != 'Q2') & (df['Release'] != 'Q1')
else:
    sys.exit("Invalid release code")
    
# select subjects that have completed all fMRI
ind = ind & ((df['fMRI_WM_Compl']== True) & (df['fMRI_Mot_Compl']==True) 
             & (df['fMRI_Lang_Compl']==True) & (df['fMRI_Emo_Compl']==True)         
             & (df['RS-fMRI_Count']==4))
                
df = df[ind]  

# check if either of the two subjects recommended for exclusion by HCP are still present
df = df[~df['Subject'].isin(['209733','528446'])]
df.index = range(df.shape[0])
print 'Selected', str(df.shape[0]), 'from the release',release
print 'Number of males is:', df[df['Gender']=='M'].shape[0]
tmpAgeRanges = sorted(df['Age'].unique())
print 'Age range is', tmpAgeRanges[0].split('-')[0], '-', tmpAgeRanges[-1].split('-')[1]

# list of all selected subjects
subjects = df['Subject']
# pull their IQ, Age, Gender
age = df['Age']
gender = df['Gender']
score = df[outScore]

### Exclusion of high-motion subjects
Further exclude subjects with >0.14 frame-to-frame head motion estimate averged across both rest runs (arbitrary threshold as in Finn et al 2015)

In [ ]:
ResultsDir = op.join(DATADIR,'Results')
if not op.isdir(ResultsDir): mkdir(ResultsDir)
ResultsDir = op.join(ResultsDir, pipelineName)
if not op.isdir(ResultsDir): mkdir(ResultsDir)
ResultsDir = op.join(ResultsDir, parcellation)
if not op.isdir(ResultsDir): mkdir(ResultsDir)

PEdirs = ['LR', 'RL']
RelRMSMean = np.zeros([len(subjects), 2])
excludeSub = list()

for iSub in range(len(subjects)):
    subject = str(subjects[iSub])
    RelRMSMeanFile = op.join(buildpath(subject, thisRun+'_zz'), 'Movement_RelativeRMS_mean.txt')
    fLR = RelRMSMeanFile.replace('zz','LR');
    fRL = RelRMSMeanFile.replace('zz','RL');
    
    if op.isfile(fLR) & op.isfile(fRL):
        with open(fLR,'r') as tmp:
            RelRMSMean[iSub,0] = float(tmp.read())
        with open(fRL,'r') as tmp:
            RelRMSMean[iSub,1] = float(tmp.read())
        print '{} {:.3f} {:.3f}'.format(subjects[iSub], RelRMSMean[iSub,0], RelRMSMean[iSub,1])
        if np.mean(RelRMSMean[iSub,:]) > 0.14:
            print subjects[iSub], ': too much motion, exclude'
            excludeSub.append(iSub)
            continue
    

    for iPEdir in range(len(PEdirs)):
        PEdir=PEdirs[iPEdir]
        fmriRun = thisRun+'_'+PEdir
        if parcellation=='shenetal_neuroimage2013':
            fmriFile = op.join(buildpath(subject,fmriRun), fmriRun+suffix+'.nii.gz')
            isCifti=0
        elif parcellation=='Glasser_Aseg_Suit':
            fmriFile = op.join(buildpath(subject,fmriRun), fmriRun+'_Atlas'+suffix+'.dtseries.nii')
            isCifti=1
        else:
            print 'Wrong parcellation code'
            return
        if not op.isfile(fmriFile):
            print str(subjects[iSub]), 'missing', fmriFile, ', exclude'
            excludeSub.append(iSub)
            continue
        
        if not (op.isfile(op.join(ResultsDir, str(subjects[iSub])+'_'+thisRun+'_'+PEdir+'.txt'))) or overwrite:
            print 'load and preprocess'
            if queue:
                # make a script to load and preprocess that file, then save as .mat
                jobDir = op.join(buildpath(str(subjects[iSub]),thisRun+'_'+PEdir),'jobs')
                if not op.isdir(jobDir): mkdir(jobDir)
                thispythonfn = '<< END\nimport sys\nsys.path.insert(0,"{}")\n'.format(getcwd())
                thispythonfn +='from runPipeline import *\nrunPipeline("{}","{}","{}")\nEND'.format(subject,fmriRun,fmriFile)
                thispythonfn += 'subject = "{}"\n'.format(subject)
                thispythonfn += 'fmriRun = "{}"\n'.format(fmriRun)
                thispythonfn += 'runPipeline("{}","{}","{}")\nEND'.format(subject,fmriRun,fmriFile)
                        
                jobName = 's{}_{}_{}_{}'.format(subjects[iSub],thisRun,PEdir, pipelineName)
                # prepare a script
                thisScript=op.join(jobDir,jobName+'.sh')
                with open(thisScript,'w') as fidw:
                    fidw.write('#!/bin/bash\n')
                    fidw.write('echo ${FSLSUBALREADYRUN}\n')
                    fidw.write('python {}'.format(thispythonfn))
                cmd='chmod 774 '+thisScript
                call(cmd,shell=True)
                # call to fnSubmitToCluster
                JobID = fnSubmitToCluster(thisScript,jobDir, jobName, '-p {} -l h_vmem=20G'.format(priority))
                joblist.append(JobID)
            else:
                runPipeline(subject, fmriRun, fmriFile)
        else:
            print subject[iSub], ' : ', PEdir, 'results already computed; skipping'

indkeep = np.setdiff1d(range(len(subjects)),excludeSub, assume_unique=True)

Wait for all processes to complete, before resuming pipeline execution.

In [ ]:
if queue:
    if len(joblist) != 0:
        while True:
            nleft = len(joblist)
            for i in range(nleft):
                myCmd = "qstat | grep ' {} '".format(joblist[i])
                isEmpty = False
                try:
                    cmdOut = check_output(myCmd, shell=True)
                except CalledProcessError as e:
                    isEmpty = True
                finally:
                    if isEmpty:
                        nleft = nleft-1
            if nleft == 0:
                break
            sleep(10)          

In [241]:
plt.scatter(score[indkeep],np.mean(RelRMSMean[indkeep,:],axis=1),c='b')
plt.scatter(score[excludeSub],np.mean(RelRMSMean[excludeSub,:],axis=1),c='r')
# fit a curve to the data using a least squares 1st order polynomial fit
z1 = np.polyfit(score[indkeep],np.mean(RelRMSMean[indkeep,:],axis=1),1)
z2 = np.polyfit(score,np.mean(RelRMSMean,axis=1),1)
p1 = np.poly1d(z1)
p2 = np.poly1d(z2)                
fit1 = p1(score[indkeep])
fit2 = p2(score)
# get the coordinates for the fit curve
c1_x = [np.min(score[indkeep]),np.max(score[indkeep])]
c1_y = p1(c1_x)
c2_x = [np.min(score),np.max(score)]
c2_y = p2(c2_x)
# plot line of best fit
plt.plot(c2_x,c2_y,'r-',label='before exclusion')
plt.plot(c1_x,c1_y,'b-',label='after exclusion')
plt.legend(loc=0)
plt.show()
rho1,p1 = stats.pearsonr(score[indkeep],np.mean(RelRMSMean[indkeep,:],axis=1))
rho2,p2 = stats.pearsonr(score,np.mean(RelRMSMean,axis=1))                         
print 'With all subjects: corr(IQ,motion) = {:.3f} (p = {:.3f})'.format(rho2,p2)
print 'After discarding high movers: corr(IQ,motion) = {:.3f} (p = {:.3f})'.format(rho1,p1)

### Correlation matrices

Correlate all pairs of node timecourses, resulting in 268x268 matrix of r-values.

Apply Fisher transform to obtain 268x268 matrix of z-scores. The LR and RL runs are never concatenated. Rather, we run the above pipeline on each run separately and average the two resulting matrices of z-scores.

In [ ]:
if parcellation=='shenetal_neuroimage2013':
    nParcels = 268
elif parcellation=='Glasser_Aseg_Suit':
    nParcels = 405
corrmats = np.zeros([nParcels,nParcels,len(indkeep)])
scores = np.zeros([len(indkeep)])
index = 0
for iSub in range(len(subjects)):
    if iSub not in excludeSub:
        PEdir=PEdirs[iPEdir] 
        tsFile_LR=op.join(ResultsDir,str(subjects[iSub])+'_'+thisRun+'_LR.txt')
        tsFile_RL=op.join(ResultsDir,str(subjects[iSub])+'_'+thisRun+'_RL.txt')
        ts_LR = np.loadtxt(tsFile_LR)
        ts_RL = np.loadtxt(tsFile_RL)
        # Fisher z transform of correlation coefficients
        corrMat_LR = np.arctanh(np.corrcoef(ts_LR,rowvar=0))
        corrMat_RL = np.arctanh(np.corrcoef(ts_RL,rowvar=0))
        np.fill_diagonal(corrMat_LR,1)
        np.fill_diagonal(corrMat_RL,1)
        corrmats[:,:,index] = (corrMat_LR + corrMat_RL)/2
        scores[index] = score[iSub]
        
results = {}
results[outMat] = corrmats
results[outScore] = scores
sio.savemat(op.join(ResultsDir,'{}_HCP_{}.mat'.format(thisRun,release)),results)